In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Set up for running selenium in Google Colab
## You don't need to run this code if you do it in Jupyter notebook, or other local Python setting
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,626 B in 1s (3,079 B/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
25 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree

In [ ]:
!pip install chromedriver-autoinstaller

In [ ]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

import time
import pandas as pd
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
import chromedriver_autoinstaller

print(selenium.__version__)

4.15.2


In [ ]:
# setup chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# set path to chromedriver as per your configuration
chromedriver_autoinstaller.install()

# set up the webdriver
driver = webdriver.Chrome(options=chrome_options)
print(driver)

<selenium.webdriver.chrome.webdriver.WebDriver (session="d1568615854652e717da79e2d67800b5")>


유플러스 : https://www.uplusumobile.com/support/csStore?tab=store&latitude=&longitude=&srchTxt=%EC%84%9C%EC%9A%B8&currentPageNo=35&lastPageNo=35&_csrf=50c769c2-220e-40e9-91bc-4a50e2d38a75#

kt : https://help.kt.com/store/KtStoreSearch.do

skt : https://www.tworld.co.kr/web/support/store/map

https://noanomal.tistory.com/entry/AttributeError-str-object-has-no-attribute-capabilities#google_vignette

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

url = "https://www.tworld.co.kr/web/support/store/map"
driver.get(url)
time.sleep(3)
place_names = []
addresses = []
phones = []

# 기본 설정
element1 = driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/div/div[2]/div[2]/div[1]/div[1]/div/div[1]/div[1]/div/a')
element1.click()
time.sleep(1)

element2 = driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/div/div[2]/div[2]/div[1]/div[1]/div/div[1]/div[1]/div/div/div/div/ul/li[2]/a')
element2.click()
time.sleep(1)

search_input = driver.find_element(By.XPATH, '//*[@id="searchText"]')
search_input.send_keys('서울')
time.sleep(1)

search_button = driver.find_element(By.XPATH, '//*[@id="searchBtn"]')
search_button.click()
time.sleep(3)

for page in range(59):
        for i in range(2):
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
                time.sleep(3)

        for i in range(1, 10):
                title_button = driver.find_element(By.XPATH, f"/html/body/div[2]/div[3]/div/div/div[1]/div/div[2]/div[2]/div[1]/div[2]/div[1]/div[{i}]/div[1]/div/a")
                place_names.append(title_button.text)

                address_element = driver.find_element(By.XPATH, f'//*[@id="store-map-layout"]/div[1]/div[{i}]/div[1]/div/div/div[3]/div')
                addresses.append(address_element.text)

                phone_element = driver.find_element(By.XPATH, f'//*[@id="store-map-layout"]/div[1]/div[{i}]/div[1]/div/div/div[4]').text
                phones.append(phone_element)

        if page < 58:

                for _ in range(2):
                        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_UP)
                        time.sleep(2)
                wait = WebDriverWait(driver, 5)
                next_button = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div[3]/div/div/div[1]/div/div[2]/div[2]/div[1]/div[2]/div[2]/button[3]')))
                next_button.click()
                time.sleep(2)
                driver.execute_script("window.scrollTo(0, 0);")
                time.sleep(2)

data = {'장소': place_names, '주소': addresses, '전화번호': phones}
df = pd.DataFrame(data)

df.to_excel('SKT.xlsx', index=False)